In [57]:
from connector.replay import Replay, State, Team, Input
from preprocessing    import remove_menu_and_pause
from preprocessing    import who_scored
from preprocessing    import game_state_to_numpy

from typing import Optional

import numpy as np

In [62]:
class Encoder(object):
    state_to_class = {
    tuple([0,0,0]):0,
    tuple([0,0,1]):1,
    
    tuple([-1,0,0]):2,
    tuple([-1,0,1]):3,
    
    tuple([0,-1,0]):4,
    tuple([0,-1,1]):5,
    
    tuple([1,0,0]):6,
    tuple([1,0,1]):7,
    
    tuple([0,1,0]):8,
    tuple([0,1,1]):9,
    
    tuple([1,1,0]):10,
    tuple([1,1,1]):11,
    
    tuple([-1,-1,0]):12,
    tuple([-1,-1,1]):13,
    
    tuple([-1,1,0]):14,
    tuple([-1,1,1]):15,
    
    tuple([1,-1,0]):16,
    tuple([1,-1,1]):17,
    }
    
    @staticmethod
    def boolarr_to_class(y : [bool]) -> int:
        def boolarr_to_dirstate(y : [bool]) -> np.array:
            result = [0,0,0]
            if y[0]: #UP key pressed
                result[0] += 1
            if y[1]: #down key pressed
                result[0] -= 1

            if y[2]: #left
                result[1] += 1
            if y[3]: #right
                result[1] -= 1

            if y[4]:
                result[2] += 1

            return np.array(result)

        return Encoder.state_to_class[tuple(boolarr_to_dirstate(y))]
    
    @staticmethod
    def class_to_boolarr(result : int) -> [bool]:
        assert(result<17)
        def dirstate_to_boolarr(result : np.array) -> [bool]:
            y = [False for x in range(0,5)]
            if result[0] == 1:
                y[0] = True #UP key pressed
            if result[0] == -1:
                y[1] = True #down key pressed
                
            if result[1] == 1: #left
                y[2] = True
            if result[1] == -1: #right
                y[3] = True
            if result[2] == 1:
                y[4] = True
            return y
        
        for dirstate, cls in a.items():    
            if result == cls:
                return dirstate
        raise ValueError("Class not found")
        
    @staticmethod
    def boolarr_to_enumarr(y: [bool]) -> [Input]:
        output = []
        if y[0] == 1:
            output.append(Input.Up)
        elif y[0] == -1:
            output.append(Input.Down)
            
        if y[1] == 1:
            output.append(Input.Left)
        elif y[1] == -1:
            output.append(Input.Right)
            
        if y[2] == 1:
            output.append(Input.Kick)
            
        return output
           
def prepare_train_data(episodes, stack_frames=4):
    # Take resultative episodes as training data
    train_x, train_y = [], []
    for episode in episodes:
        scorer = who_scored(episode)
        if scorer is not None:
            if scorer == Team.Blue:
                transform_blue_pivotal(episode)
            x, y = zip(*[game_state_to_numpy(e, scorer) for e in episode])

            # If there was no target player - ignore
            if y is not None:
                train_x.append(x)
                train_y.append(y)
                
    # Stack n-frames
    x = []
    y = []
    for episode_i in range(len(train_x)):
        episode_x      = train_x[episode_i]
        episode_y      = train_y[episode_i]
        
        # Stacked samples
        train_sample = [np.concatenate(episode_x[0]).ravel() for _ in range(stack_frames)]
        
        x.append(np.concatenate(train_sample).ravel())
        y.append(Encoder.boolarr_to_class(np.array(episode_y[0])))
        
        for frame_i in range(1, len(episode_x)):
            frame_x = episode_x[frame_i]
            frame_y = episode_y[frame_i]
            
            # Move stack
            for i in range(1, stack_frames):
                train_sample[i] = train_sample[i - 1]
            train_sample[0] = np.concatenate(frame_x).ravel()
            
            x.append(np.concatenate(train_sample).ravel())
            y.append(Encoder.boolarr_to_class(np.array(frame_y)))
            
    return x, y

In [59]:
def transform_blue_pivotal(frames) -> Replay:
    """
    Inplace transformation of disk positions and velocities based on the pivotal team side.
    Inverse x-axis basically.
    
    frames: a list of replay frames
    """
    for frame in frames:
        if frame.players == None:
            continue
            
        for player in frame.players:
            # Inverse position and velocity along x-axis
            player.disc.x  = -player.disc.x
            player.disc.vx = -player.disc.vx
            
            # Swap left and right actions
            temp            = player.input[2]
            player.input[2] = player.input[3]
            player.input[3] = temp
            
        # Inverse position and velocity of the ball along x-axis
        frame.ball.x  = -frame.ball.x
        frame.ball.vx = -frame.ball.vx

In [63]:
# Parse replay and etract game epsiodes
names, replay = Replay(open("replays/cups/edition7/2018-12-02.bin", mode="rb").read())
episodes = remove_menu_and_pause(replay)
t1, t2 = prepare_train_data(episodes, 1)

In [ ]:
import lightgbm as lgb

In [ ]:
imitator = lgb.LGBMClassifier()